# 1.0 Libary and parameter settings

In [ ]:
## 1.0.1 Library import
import configparser

from azureml.core import Workspace
from azureml.core.authentication import AzureCliAuthentication
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.pipeline.core._restclients.aeva.models.error_response import ErrorResponseException
from azureml.pipeline.core import Pipeline, PipelineEndpoint, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep

In [ ]:
## 1.0.2 Retrieve configuration file
config_ini = configparser.ConfigParser()
config_ini.read('./common/config.ini', encoding='utf-8')

## 1.0.3 Basic Azure parameters
subscription_id = config_ini.get('Azure', 'subscription_id')
resource_group = config_ini.get('Azure', 'resource_group')
workspace_name = config_ini.get('Azure', 'workspace_name')

## 1.0.4 Data
train_ratio = config_ini.get('data', 'train_ratio')
dataset_name = config_ini.get('data', 'dataset_name')
dataset_name_for_train = config_ini.get('data', 'dataset_name_for_train')
dataset_name_for_test = config_ini.get('data', 'dataset_name_for_test')

## 1.0.5 Azure ML parameters
cluster_name = config_ini.get('AML', 'cluster_name')
vm_size = config_ini.get('AML', 'vm_size')
vm_location = config_ini.get('AML', 'vm_location')
managed_id = config_ini.get('AML', 'managed_id')
image_analysis_type = config_ini.get('AML', 'image_analysis_type')
random_seed = config_ini.get('AML', 'random_seed')

## 1.0.6 Parameters for image classification
experiment_name = config_ini.get('AML', 'experiment_name')
base_model = config_ini.get('AML', 'base_model')
pipelineName = config_ini.get('AML', 'pipelineName')
model_name = config_ini.get('AML', 'model_name')

# 1.1 Authentication

In [ ]:
## 1.1.1 Retrieve AML workspace with CLI authentication
cli_auth = AzureCliAuthentication()
ws = Workspace(subscription_id=subscription_id,
               resource_group=resource_group,
               workspace_name=workspace_name,
               auth=cli_auth)

# 1.2 Define compute target

`compute_target` will be used in training with given image files under GPU-clusters.

Also, you need `NC`-series VM instead of `NV`-series for automl library for images.

In [ ]:
## 1.2.1 Define computer target
try:
    compute_target = ws.compute_targets[cluster_name]
    print("Found existing compute target.")
except KeyError:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        idle_seconds_before_scaledown=600,
        min_nodes=0,
        max_nodes=4,
        location=vm_location,
        identity_type=managed_id,
    )
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(
    show_output=True, min_node_count=None, timeout_in_minutes=20
)

# 1.3 Run-Configuration

## 1.3.1 Run configuration

It connects `compute_target` with the other settings in executing AML pipeline.

In [ ]:
aml_run_config = RunConfiguration()
aml_run_config.target = compute_target

## 1.3.2 Add some packages for python environment

Ideally, we want to use the same environment through training and infering processes.

We referred [this environment](https://ml.azure.com/registries/azureml/environments/AzureML-AutoML-DNN-Vision-GPU/version/96) as basic idea to pick up which library and version are appropriate to deploy. 

You can find the more about the [package dependency](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py#remarks).

Note:
- `azureml-automl-core`: The basic library in using `automl`.
- `azureml-automl-dnn-vision==1.43.0`: We need this library to infer image files with the deep-learning model generated by `automl`.
    - Associated to this library, these could be candidates to get successful results in AML pipelines:
        - `python` version as `3.7`
        - `numpy`, whose version as `1.20.1`
        - `pycocotools`, whose version as `2.0.2`
- You can explore the appropriate combination, in case that you need more.

In [ ]:
aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    python_version='3.7'
    ,conda_packages=['pandas'
                ,'scikit-learn'
                ,'numpy==1.20.1'
                ,'pycocotools==2.0.2'
                ]
    ,pip_packages=['azureml-sdk'
                ,'azureml-automl-core'
                ,'azureml-automl-dnn-vision==1.43.0'
                ]
    ,pin_sdk_version=False)

# 1.4 Generate AML pipeline for training

In [ ]:
## 1.4.1 Define python script and its arguments to be used
pipeline_param01 = PipelineParameter(name="pipeline_arg01", default_value="dummy_subscription")
pipeline_param02 = PipelineParameter(name="pipeline_arg02", default_value="dummy_resource_group")
pipeline_param03 = PipelineParameter(name="pipeline_arg03", default_value="dummy_work_space")
pipeline_param04 = PipelineParameter(name="pipeline_arg04", default_value="cluster_abc")
pipeline_param05 = PipelineParameter(name="pipeline_arg05", default_value="image_classification_experiment")
pipeline_param06 = PipelineParameter(name="pipeline_arg06", default_value="base_model")
pipeline_param07 = PipelineParameter(name="pipeline_arg07", default_value="dataset")
pipeline_param08 = PipelineParameter(name="pipeline_arg08", default_value="1")
pipeline_param09 = PipelineParameter(name="pipeline_arg09", default_value="1")
pipeline_param10 = PipelineParameter(name="pipeline_arg10", default_value="0.5")
pipeline_param11 = PipelineParameter(name="pipeline_arg11", default_value="1")
pipeline_param12 = PipelineParameter(name="pipeline_arg12", default_value="image_classification_model")
pipeline_param13 = PipelineParameter(name="pipeline_arg13", default_value="image_classification")

trainStep = PythonScriptStep(
    script_name="train.py",
    arguments=[
        "--subscription_id",         pipeline_param01,
        "--resource_group",          pipeline_param02,
        "--workspace_name",          pipeline_param03,
        "--cluster_name",            pipeline_param04,
        "--experiment_name",         pipeline_param05,
        "--base_model",              pipeline_param06,
        "--dataset_name",            pipeline_param07,
        "--dataset_name_for_train",  pipeline_param08,
        "--dataset_name_for_test",   pipeline_param09,
        "--train_ratio",             pipeline_param10,
        "--random_seed",             pipeline_param11,
        "--model_name",              pipeline_param12,
        "--image_analysis_type",     pipeline_param13
    ],
    compute_target=compute_target,
    source_directory='.',
    runconfig=aml_run_config,
    allow_reuse = True,
)

## 1.4.2.1 Define AML pipeline

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[trainStep])

## 1.4.2.2 Work-around in some error

Try the following commands in the next cell, if you encounter the error:

```sh
While attempting to take snapshot of .
Your total snapshot size exceeds the limit of 300.0 MB
```

ref. https://docs.microsoft.com/en-us/azure/machine-learning/how-to-save-write-experiment-files#storage-limits-of-experiment-snapshots

In [ ]:
import azureml
azureml._restclient.snapshots_client.SNAPSHOT_MAX_SIZE_BYTES = 1000000000

## 1.4.3 Publish the pipeline

Publish the pipeline, and you can use it from other Azure resources like [Azure Data Factory](https://azure.microsoft.com/en-us/services/data-factory/).
Also, by using the following steps, the published URI will be kept.

In [ ]:
try:
    pipelineEndpoint = PipelineEndpoint.get(workspace=ws, name=pipelineName)
except ErrorResponseException as ex:
    if "not found in workspace" in ex.message:
        pipelineEndpoint = None
    else:
        raise

if pipelineEndpoint is None:
    print('Pipeline does not exists, creating new: ' + pipelineName)
    pipelineEndpoint = PipelineEndpoint.publish(workspace = ws
                                        ,name = pipelineName
                                        ,pipeline=pipeline
                                        ,description="My Published Pipeline Description.")
else:
    print('Found existing pipeline ' + pipelineName + ', adding new version.')
    published_pipeline = pipeline.publish(name = pipelineName + "_Pipeline")
    pipelineEndpoint.add_default(published_pipeline)

## 1.4.4 Submit the pipeline

You can execute the pipeline by giving the parameters.

In [ ]:
pipeline.submit(experiment_name=experiment_name,
                pipeline_parameters={"pipeline_arg01": subscription_id
                                    ,"pipeline_arg02": resource_group
                                    ,"pipeline_arg03": workspace_name
                                    ,"pipeline_arg04": cluster_name
                                    ,"pipeline_arg05": experiment_name
                                    ,"pipeline_arg06": base_model
                                    ,"pipeline_arg07": dataset_name
                                    ,"pipeline_arg08": dataset_name_for_train
                                    ,"pipeline_arg09": dataset_name_for_test
                                    ,"pipeline_arg10": train_ratio
                                    ,"pipeline_arg11": random_seed
                                    ,"pipeline_arg12": model_name
                                    ,"pipeline_arg13": image_analysis_type                             
                                    })